# Leitura das Bibliotecas utilizadas no projeto

In [ ]:
# Análise e tratamento de dados
import pandas as pd
import numpy as np
import json

# Requisição via API
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Banco de dados
import sqlite3

# Visualização dos dados
import matplotlib.pyplot as plt

# Regressão Linear
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Etapa 01 - Acessar dados sobre a Receita Municipal

## Requisitar dados do site do Tribunal de Contas do Estado de Pernambuco

In [ ]:
# Obs: etapa inicial, preparatória e necessária para não ter dificuldades com requisições ao site do TCE-PE

session = requests.Session()
retry = Retry(connect=5, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [ ]:
# Solicita dados do site do TCE-PE via API
exercicios = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']
dicionario = {}
base_dados = []
base_dados_temp = []
for i, ano in enumerate(exercicios):
  url = 'https://sistemas.tce.pe.gov.br/DadosAbertos/ReceitasMunicipais!json?ANOREFERENCIA=%s&ID_UNIDADE_GESTORA=319' % (ano)
  resposta = session.get(url, stream=True)
  if i == 0:
    dicionario = json.loads(resposta.text)
    base_dados = pd.DataFrame.from_dict(data=dicionario['resposta']['conteudo'])
    print('Ano {} concluído.'.format(ano))
    print(f'Ano {ano}: {base_dados.shape[0]} linhas carregadas.')
  else:
    dicionario = json.loads(resposta.text)
    base_dados_temp = pd.DataFrame.from_dict(data=dicionario['resposta']['conteudo'])
    print('Ano {} concluído.'.format(ano))
    print('\n')
    print(f'Ano {ano}: {base_dados_temp.shape[0]} linhas carregadas.')
    base_dados = pd.concat([base_dados, base_dados_temp])
    print(f'Total: {base_dados.shape[0]} linhas carregadas.')
    

Ano 2012 concluído.
Ano 2012: 890 linhas carregadas.
Ano 2013 concluído.


Ano 2013: 934 linhas carregadas.
Total: 1824 linhas carregadas.
Ano 2014 concluído.


Ano 2014: 972 linhas carregadas.
Total: 2796 linhas carregadas.
Ano 2015 concluído.


Ano 2015: 1248 linhas carregadas.
Total: 4044 linhas carregadas.
Ano 2016 concluído.


Ano 2016: 1376 linhas carregadas.
Total: 5420 linhas carregadas.
Ano 2017 concluído.


Ano 2017: 1305 linhas carregadas.
Total: 6725 linhas carregadas.
Ano 2018 concluído.


Ano 2018: 1347 linhas carregadas.
Total: 8072 linhas carregadas.
Ano 2019 concluído.


Ano 2019: 1428 linhas carregadas.
Total: 9500 linhas carregadas.
Ano 2020 concluído.


Ano 2020: 1892 linhas carregadas.
Total: 11392 linhas carregadas.
Ano 2021 concluído.


Ano 2021: 1902 linhas carregadas.
Total: 13294 linhas carregadas.
Ano 2022 concluído.


Ano 2022: 649 linhas carregadas.
Total: 13943 linhas carregadas.


## Análise e Tratamento dos dados requisitados

In [ ]:
base_dados.shape

(13943, 12)

In [ ]:
base_dados.columns

Index(['ID_UNIDADE_GESTORA', 'NUMERORECEITAORCAMENTARIAUG', 'DESCRICAO',
       'CATEGORIARECEITA', 'ANOREFERENCIA', 'MESREFERENCIA', 'VALORLANCAMENTO',
       'ESFERA', 'ORIGEMRECEITA', 'DEDUCAORECEITA', 'ESPECIERECEITA',
       'DESCRICAOFONTE'],
      dtype='object')

In [ ]:
base_dados = base_dados.reset_index()

In [ ]:
base_dados = base_dados.drop(labels='index', axis=1)

In [ ]:
base_dados['COMPETENCIA'] = pd.to_datetime({'year': base_dados['ANOREFERENCIA'], 'month': base_dados['MESREFERENCIA'], 'day': 1}, format="%m/%Y")

In [ ]:
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13943 entries, 0 to 13942
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID_UNIDADE_GESTORA           13943 non-null  object        
 1   NUMERORECEITAORCAMENTARIAUG  13941 non-null  object        
 2   DESCRICAO                    13941 non-null  object        
 3   CATEGORIARECEITA             13943 non-null  object        
 4   ANOREFERENCIA                13943 non-null  object        
 5   MESREFERENCIA                13943 non-null  object        
 6   VALORLANCAMENTO              13943 non-null  object        
 7   ESFERA                       13943 non-null  object        
 8   ORIGEMRECEITA                13941 non-null  object        
 9   DEDUCAORECEITA               13943 non-null  object        
 10  ESPECIERECEITA               12484 non-null  object        
 11  DESCRICAOFONTE               12119 non-nu

In [ ]:
base_dados.head()

,ID_UNIDADE_GESTORA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,CATEGORIARECEITA,ANOREFERENCIA,MESREFERENCIA,VALORLANCAMENTO,ESFERA,ORIGEMRECEITA,DEDUCAORECEITA,ESPECIERECEITA,DESCRICAOFONTE,COMPETENCIA
0,319,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,Dedução de Receitas,2012,2,-3655348.80,M,Alienação de Bens,9,NaN,NaN,2012-02-01
1,319,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,Dedução de Receitas,2012,4,-2974700.38,M,Alienação de Bens,9,NaN,NaN,2012-04-01
2,319,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,Dedução de Receitas,2012,5,-4365097.58,M,Alienação de Bens,9,NaN,NaN,2012-05-01
3,319,917220102,Dedução de Cota-Parte do IPVA - FUNDEB,Dedução de Receitas,2012,8,-3891.64,M,Alienação de Bens,9,NaN,NaN,2012-08-01
4,319,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,Dedução de Receitas,2012,9,-7644.57,M,Alienação de Bens,9,NaN,NaN,2012-09-01


In [ ]:
# Lista de Receitas e Códigos
receitas_cod = base_dados[['NUMERORECEITAORCAMENTARIAUG', 'DESCRICAO']].drop_duplicates()
receitas_cod.to_csv('receitas_cod.csv', sep=';', index=False)

In [ ]:
base_dados.loc[base_dados['NUMERORECEITAORCAMENTARIAUG'].isnull()]

,ID_UNIDADE_GESTORA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,CATEGORIARECEITA,ANOREFERENCIA,MESREFERENCIA,VALORLANCAMENTO,ESFERA,ORIGEMRECEITA,DEDUCAORECEITA,ESPECIERECEITA,DESCRICAOFONTE,COMPETENCIA


In [ ]:
base_dados.drop(base_dados[base_dados['NUMERORECEITAORCAMENTARIAUG'].isnull()].index, inplace=True)

In [ ]:
base_dados.to_csv('/content/base_dados')

In [ ]:
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941 entries, 0 to 13942
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID_UNIDADE_GESTORA           13941 non-null  object        
 1   NUMERORECEITAORCAMENTARIAUG  13941 non-null  object        
 2   DESCRICAO                    13941 non-null  object        
 3   CATEGORIARECEITA             13941 non-null  object        
 4   ANOREFERENCIA                13941 non-null  object        
 5   MESREFERENCIA                13941 non-null  object        
 6   VALORLANCAMENTO              13941 non-null  object        
 7   ESFERA                       13941 non-null  object        
 8   ORIGEMRECEITA                13941 non-null  object        
 9   DEDUCAORECEITA               13941 non-null  object        
 10  ESPECIERECEITA               12484 non-null  object        
 11  DESCRICAOFONTE               12117 non-nu

In [ ]:
# Transformar numeros em valores flutuantes
base_dados['VALORLANCAMENTO'] = base_dados['VALORLANCAMENTO'].astype(float)
base_dados['NUMERORECEITAORCAMENTARIAUG'] = base_dados['NUMERORECEITAORCAMENTARIAUG'].astype(int)

In [ ]:
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941 entries, 0 to 13942
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID_UNIDADE_GESTORA           13941 non-null  object        
 1   NUMERORECEITAORCAMENTARIAUG  13941 non-null  int64         
 2   DESCRICAO                    13941 non-null  object        
 3   CATEGORIARECEITA             13941 non-null  object        
 4   ANOREFERENCIA                13941 non-null  object        
 5   MESREFERENCIA                13941 non-null  object        
 6   VALORLANCAMENTO              13941 non-null  float64       
 7   ESFERA                       13941 non-null  object        
 8   ORIGEMRECEITA                13941 non-null  object        
 9   DEDUCAORECEITA               13941 non-null  object        
 10  ESPECIERECEITA               12484 non-null  object        
 11  DESCRICAOFONTE               12117 non-nu

## Gravar dados requisitados em um Banco de Dados

In [ ]:
base_dados_gravar = base_dados[['COMPETENCIA', 'ANOREFERENCIA', 'ORIGEMRECEITA', 'DEDUCAORECEITA', 'NUMERORECEITAORCAMENTARIAUG', 'DESCRICAO', 'VALORLANCAMENTO']]

In [ ]:
base_dados_gravar.head()

,COMPETENCIA,ANOREFERENCIA,ORIGEMRECEITA,DEDUCAORECEITA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,VALORLANCAMENTO
0,2012-02-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-3655348.80
1,2012-04-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-2974700.38
2,2012-05-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-4365097.58
3,2012-08-01,2012,Alienação de Bens,9,917220102,Dedução de Cota-Parte do IPVA - FUNDEB,-3891.64
4,2012-09-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-7644.57


In [ ]:
base_dados_gravar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941 entries, 0 to 13942
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   COMPETENCIA                  13941 non-null  datetime64[ns]
 1   ANOREFERENCIA                13941 non-null  object        
 2   ORIGEMRECEITA                13941 non-null  object        
 3   DEDUCAORECEITA               13941 non-null  object        
 4   NUMERORECEITAORCAMENTARIAUG  13941 non-null  int64         
 5   DESCRICAO                    13941 non-null  object        
 6   VALORLANCAMENTO              13941 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 871.3+ KB


In [ ]:
# Transformar data em texto para armazenamento no banco de dados - hipótese de explicação do erro.
base_dados_gravar['COMPETENCIA'] = base_dados_gravar['COMPETENCIA'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Criar conexão
con = sqlite3.connect('baseTransferencias.db')

# Criar cursor
cur = con.cursor()

In [ ]:
#Inserir DE-PARA receitas no banco de dados - Fazer upload do arquivo.
receitas_tratadas = pd.read_csv('/content/receitas_tratadas.csv', sep=';', encoding='ISO-8859–1')

In [ ]:
receitas_tratadas.head()

,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,CLASSIFICACAO_SEREC,DETALHAMENTO
0,917220101,Deducao de Cota-Parte do ICMS - FUNDEB,ICMS,DEDUCAO FUNDEB
1,917220102,Deducao de Cota-Parte do IPVA - FUNDEB,IPVA,DEDUCAO FUNDEB
2,917220104,Deducao de Cota-Parte do IPI sobre Exportacao ...,IPI ExportaCAo,DEDUCAO FUNDEB
3,917230000,Deducao de Transferencias dos Municipios,OUTRAS TRANSFERENCIAS,OUTRAS DEDUCOES
4,917213600,Deducao de Transferencia Financeira do ICMS De...,ICMS DesoneraCAo L.C. Nº 87.96,DEDUCAO FUNDEB


In [ ]:
#Criar tabela auxiliar
cur.execute('''CREATE TABLE receitasDePara
               (numeroReceitaOrcamentariaUG_2 text,
               classificacaoSerec text,
               detalhamento text
               )
               ''')

In [ ]:
#Inserir dados na tabela
receitas_tratadas_normalizadas = receitas_tratadas[['NUMERORECEITAORCAMENTARIAUG',	
                                                    'CLASSIFICACAO_SEREC', 
                                                    'DETALHAMENTO']]
receitas_tratadas_lista = receitas_tratadas_normalizadas.values.tolist()

try:
  cur.executemany("INSERT INTO receitasDePara (numeroReceitaOrcamentariaUG_2, classificacaoSerec, detalhamento) VALUES (?, ?, ?)", receitas_tratadas_lista)
  con.commit()
  print("Dados inseridos com sucesso.")
except Exception as e:
  print(str(e))
  print("Falha na inserção de dados.")
finally:
  pass

Dados inseridos com sucesso.


In [ ]:
#Criar tabela principal
cur.execute('''CREATE TABLE transferencias
               (data text,
               ano text,
               origemReceita text, 
               deducaoReceita text,
               numeroReceitaOrcamentariaUG text,
               descricao text,
               valor real,
               FOREIGN KEY (numeroReceitaOrcamentariaUG) REFERENCES receitaDePara(numeroReceitaOrcamentariaUG_2))
               ''')

In [ ]:
#Inserir dados na tabela
base_dados_lista = base_dados_gravar.values.tolist()

try:
  cur.executemany("INSERT INTO transferencias (data, ano, origemReceita, deducaoReceita, numeroReceitaOrcamentariaUG, descricao, valor) VALUES (?, ?, ?, ?, ?, ?, ?)", base_dados_lista)
  con.commit()
  print("Dados inseridos com sucesso.")
except Exception as e:
  print(str(e))
  print("Falha na inserção de dados.")
finally:
  con.close()

Dados inseridos com sucesso.


# ETAPA 02 - Organização dos dados para Previsão anual

## Salvar tabelas em variáveis

In [ ]:
# Criar nova conexão e consultar tabelas gravadas no Banco de dados 
con = sqlite3.connect('baseTransferencias.db')
cur = con.cursor()

In [ ]:
# Tabela Principal
cur.execute("SELECT * FROM transferencias")
tabela_principal = cur.fetchall()
tabela_principal = pd.DataFrame(base_dados)
tabela_principal.columns = base_dados_gravar.columns

In [ ]:
# Tabela Auxiliar
cur.execute("SELECT * FROM receitasDePara")
tabela_auxiliar = cur.fetchall()
tabela_auxiliar = pd.DataFrame(tabela_auxiliar)
tabela_auxiliar.columns = receitas_tratadas_normalizadas.columns

In [ ]:
tabela_principal

,COMPETENCIA,ANOREFERENCIA,ORIGEMRECEITA,DEDUCAORECEITA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,VALORLANCAMENTO
0,2012-02-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-3655348.80
1,2012-04-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-2974700.38
2,2012-05-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-4365097.58
3,2012-08-01,2012,Alienação de Bens,9,917220102,Dedução de Cota-Parte do IPVA - FUNDEB,-3891.64
4,2012-09-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-7644.57
...,...,...,...,...,...,...,...
13936,2022-03-01,2022,Transferências Correntes,0,17215001,Cota-Parte do ICMS - Principal,33651308.01
13937,2022-04-01,2022,Transferências Correntes,0,17299901,Outras Transferências dos Estados - Principal,10242.82
13938,2022-04-01,2022,Transferências Correntes,0,17215101,Cota-Parte do IPVA - Principal,9189377.74
13939,2022-04-01,2022,Operações de Crédito,0,21199901,Outras Operações de Crédito - Mercado Interno ...,5892664.46


In [ ]:
tabela_auxiliar

,NUMERORECEITAORCAMENTARIAUG,CLASSIFICACAO_SEREC,DETALHAMENTO
0,917220101,ICMS,DEDUCAO FUNDEB
1,917220102,IPVA,DEDUCAO FUNDEB
2,917220104,IPI ExportaCAo,DEDUCAO FUNDEB
3,917230000,OUTRAS TRANSFERENCIAS,OUTRAS DEDUCOES
4,917213600,ICMS DesoneraCAo L.C. Nº 87.96,DEDUCAO FUNDEB
...,...,...,...
287,17149901,None,None
288,17165011,None,None
289,13110203,None,None
290,17179901,None,None


In [ ]:
con.close()

## Juntar tabelas e filtrar apenas Receita com Fundo de Participação dos Municípios

In [ ]:
tabela_completa = None
tabela_completa = base_dados.merge(tabela_auxiliar, how='left', on='NUMERORECEITAORCAMENTARIAUG')

In [ ]:
tabela_completa.shape

(15210, 9)

In [ ]:
tabela_completa.head()

,COMPETENCIA,ANOREFERENCIA,ORIGEMRECEITA,DEDUCAORECEITA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,VALORLANCAMENTO,CLASSIFICACAO_SEREC,DETALHAMENTO
0,2012-02-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-3655348.80,ICMS,DEDUCAO FUNDEB
1,2012-04-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-2974700.38,ICMS,DEDUCAO FUNDEB
2,2012-05-01,2012,Alienação de Bens,9,917220101,Dedução de Cota-Parte do ICMS - FUNDEB,-4365097.58,ICMS,DEDUCAO FUNDEB
3,2012-08-01,2012,Alienação de Bens,9,917220102,Dedução de Cota-Parte do IPVA - FUNDEB,-3891.64,IPVA,DEDUCAO FUNDEB
4,2012-09-01,2012,Alienação de Bens,9,917220104,Dedução de Cota-Parte do IPI sobre Exportação ...,-7644.57,IPI ExportaCAo,DEDUCAO FUNDEB


In [ ]:
# Filtrar receita FPM
tabela_receita_FPM = tabela_completa.loc[tabela_completa['CLASSIFICACAO_SEREC'] == "FPM"]

In [ ]:
tabela_receita_FPM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314 entries, 494 to 15178
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COMPETENCIA                  314 non-null    object 
 1   ANOREFERENCIA                314 non-null    object 
 2   ORIGEMRECEITA                314 non-null    object 
 3   DEDUCAORECEITA               314 non-null    object 
 4   NUMERORECEITAORCAMENTARIAUG  314 non-null    object 
 5   DESCRICAO                    314 non-null    object 
 6   VALORLANCAMENTO              314 non-null    float64
 7   CLASSIFICACAO_SEREC          314 non-null    object 
 8   DETALHAMENTO                 314 non-null    object 
dtypes: float64(1), object(8)
memory usage: 24.5+ KB


In [ ]:
# Transformar COMPETENCIA para Datetime
tabela_receita_FPM['COMPETENCIA'] = pd.to_datetime(tabela_receita_FPM['COMPETENCIA'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
tabela_receita_FPM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314 entries, 494 to 15178
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   COMPETENCIA                  314 non-null    datetime64[ns]
 1   ANOREFERENCIA                314 non-null    object        
 2   ORIGEMRECEITA                314 non-null    object        
 3   DEDUCAORECEITA               314 non-null    object        
 4   NUMERORECEITAORCAMENTARIAUG  314 non-null    object        
 5   DESCRICAO                    314 non-null    object        
 6   VALORLANCAMENTO              314 non-null    float64       
 7   CLASSIFICACAO_SEREC          314 non-null    object        
 8   DETALHAMENTO                 314 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 24.5+ KB


In [ ]:
tabela_receita_FPM.set_index('COMPETENCIA', inplace=True)
tabela_receita_FPM.head()

,ANOREFERENCIA,ORIGEMRECEITA,DEDUCAORECEITA,NUMERORECEITAORCAMENTARIAUG,DESCRICAO,VALORLANCAMENTO,CLASSIFICACAO_SEREC,DETALHAMENTO
COMPETENCIA,,,,,,,,
2012-03-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,4772635.69,FPM,PRINCIPAL
2012-03-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,4772635.69,FPM,PRINCIPAL
2012-01-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,5809328.85,FPM,PRINCIPAL
2012-01-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,5809328.85,FPM,PRINCIPAL
2012-02-01,2012,Transferências Correntes,0,17210102,Cota-Parte do Fundo de Participação dos Municí...,7027265.76,FPM,PRINCIPAL


In [ ]:
tabela_receita_FPM.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 314 entries, 2012-03-01 to 2022-02-01
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ANOREFERENCIA                314 non-null    object 
 1   ORIGEMRECEITA                314 non-null    object 
 2   DEDUCAORECEITA               314 non-null    object 
 3   NUMERORECEITAORCAMENTARIAUG  314 non-null    object 
 4   DESCRICAO                    314 non-null    object 
 5   VALORLANCAMENTO              314 non-null    float64
 6   CLASSIFICACAO_SEREC          314 non-null    object 
 7   DETALHAMENTO                 314 non-null    object 
dtypes: float64(1), object(7)
memory usage: 22.1+ KB


In [ ]:
tabela_receita_FPM = tabela_receita_FPM.drop_duplicates()

In [ ]:
tabela_receita_FPM.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 242 entries, 2012-03-01 to 2022-02-01
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ANOREFERENCIA                242 non-null    object 
 1   ORIGEMRECEITA                242 non-null    object 
 2   DEDUCAORECEITA               242 non-null    object 
 3   NUMERORECEITAORCAMENTARIAUG  242 non-null    object 
 4   DESCRICAO                    242 non-null    object 
 5   VALORLANCAMENTO              242 non-null    float64
 6   CLASSIFICACAO_SEREC          242 non-null    object 
 7   DETALHAMENTO                 242 non-null    object 
dtypes: float64(1), object(7)
memory usage: 17.0+ KB


In [ ]:
# Gerar tabela para validação dos dados em excel
tabela_receita_FPM.to_csv('/content/tabela_receita_FPM.csv')

## Juntar dados em bases anuais

In [ ]:
tabela_receita_FPM_anual = tabela_receita_FPM.resample('A').sum()

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
tabela_receita_FPM_anual_base = tabela_receita_FPM_anual.loc["2012":"2021"]

In [ ]:
tabela_receita_FPM_anual_base["ANO"] = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# ETAPA 03 - Previsão para mês posterior

In [ ]:
# Divisão em Treino e Teste


In [ ]:
lr = LinearRegression()

In [ ]:
model = lr.fit(X = tabela_receita_FPM_anual['ANO'], y = tabela_receita_FPM_anual['VALORLANCAMENTO'])

KeyError: ignored

In [ ]:

# Confirmar valor com arrecadação Município
# Consolidar por ano
# Aplicar modelos > função?